In [140]:
# Importing Libraries
import pandas as pd
import json
import re
import os
from moviepy import VideoFileClip
from tqdm import tqdm
import yaml

### Data Prep Helper Functions

In [125]:
# Function to summarize dataset structure
def summarize_dataset_structure(base_dir="."):
    """
    Summarize how many videos exist in each class folder across train/val/test splits.
    Handles nested folder structures (e.g., class -> video -> clips).

    Example layout:
        base_dir/
            train/
                Abuse/
                    Abuse001_x264/
                        Abuse001_x264_0.mp4
                        Abuse001_x264_1.mp4
                Shoplifting/
            val/
            test/
    """
    for split in ["train", "val", "test"]:
        split_path = os.path.join(base_dir, split)
        if not os.path.exists(split_path):
            print(f"⚠️ Split folder not found: {split_path}")
            continue

        print(f"\n📂 Stats for split: {split}")
        total_videos = 0

        # Iterate over each class folder (Abuse, Shoplifting, etc.)
        for cls in sorted(os.listdir(split_path)):
            cls_path = os.path.join(split_path, cls)
            if not os.path.isdir(cls_path):
                continue

            video_count = 0

            # Go into subdirectories (e.g., Abuse001_x264)
            for root, _, files in os.walk(cls_path):
                video_files = [
                    f for f in files
                    if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv', '.npy'))
                ]
                video_count += len(video_files)

            print(f"  🗂️ {cls}: {video_count} video clips")
            total_videos += video_count

        print(f"  ➤ Total video clips in '{split}': {total_videos}")

In [126]:
# Function to load UCF Crime-style JSON annotations
def load_ucf_json(json_path):
    """
    Loads a UCF Crime-style JSON annotation file and converts it
    into a flattened pandas DataFrame.

    Args:
        json_path (str): Path to the JSON annotation file.
    
    Returns:
        pd.DataFrame: Flattened DataFrame with columns:
            ['video', 'duration', 'start', 'end', 'description']
    """
    with open(json_path, "r") as f:
        data = json.load(f)

    rows = []
    for video, info in data.items():
        for (ts, sentence) in zip(info.get("timestamps", []), info.get("sentences", [])):
            rows.append({
                "video": video,
                "duration": info.get("duration", None),
                "start": ts[0],
                "end": ts[1],
                "description": sentence
            })

    df = pd.DataFrame(rows)
    return df

In [132]:
def add_clip_paths(df):
    """
    Adds a 'clip_path' column based only on DataFrame columns:
    folder/video_basename/video_basename_i.mp4

    Args:
        df (pd.DataFrame): must contain 'folder' and 'video' columns.

    Returns:
        pd.DataFrame: with an extra column 'clip_path'
    """
    df = df.copy()

    # Remove .mp4 extension from video to get folder/video base
    df["video_base"] = df["video"].str.replace(".mp4", "", regex=False)

    # Get index per unique (folder, video) group — i = 0, 1, 2, ...
    df["clip_idx"] = df.groupby(["folder", "video"]).cumcount()

    # Construct the relative path
    df["clip_path"] = df.apply(
        lambda row: f"{row['folder']}/{row['video_base']}/{row['video_base']}_{row['clip_idx']}.mp4",
        axis=1
    )

    df.drop(['video_base', 'clip_idx'], axis=1, inplace=True)

    return df

### Loading UCA Annotations

In [133]:
train_file = "../Surveillance-Video-Understanding-main/ucf-annotation/json/UCFCrime_Train.json"
test_file = "../Surveillance-Video-Understanding-main/ucf-annotation/json/UCFCrime_Test.json"
val_file = "../Surveillance-Video-Understanding-main/ucf-annotation/json/UCFCrime_Val.json"

In [134]:
train_df = load_ucf_json(train_file)
test_df = load_ucf_json(test_file)
val_df = load_ucf_json(val_file)

In [135]:
train_df['folder'] = train_df['video'].apply(lambda x: re.match(r'([A-Za-z]+)', x).group(1) if re.match(r'([A-Za-z]+)', x) else None)
test_df['folder'] = test_df['video'].apply(lambda x: re.match(r'([A-Za-z]+)', x).group(1) if re.match(r'([A-Za-z]+)', x) else None)
val_df['folder'] = val_df['video'].apply(lambda x: re.match(r'([A-Za-z]+)', x).group(1) if re.match(r'([A-Za-z]+)', x) else None)

In [136]:
train_df['video'] = train_df['video']+".mp4"
test_df['video'] = test_df['video']+".mp4"
val_df['video'] = val_df['video']+".mp4"

In [137]:
train_df = add_clip_paths(train_df)
train_df[train_df["video"] == "Abuse001_x264.mp4"][["folder", "video", "clip_path"]]

,folder,video,clip_path
0,Abuse,Abuse001_x264.mp4,Abuse/Abuse001_x264/Abuse001_x264_0.mp4
1,Abuse,Abuse001_x264.mp4,Abuse/Abuse001_x264/Abuse001_x264_1.mp4
2,Abuse,Abuse001_x264.mp4,Abuse/Abuse001_x264/Abuse001_x264_2.mp4
3,Abuse,Abuse001_x264.mp4,Abuse/Abuse001_x264/Abuse001_x264_3.mp4
4,Abuse,Abuse001_x264.mp4,Abuse/Abuse001_x264/Abuse001_x264_4.mp4
5,Abuse,Abuse001_x264.mp4,Abuse/Abuse001_x264/Abuse001_x264_5.mp4
6,Abuse,Abuse001_x264.mp4,Abuse/Abuse001_x264/Abuse001_x264_6.mp4
7,Abuse,Abuse001_x264.mp4,Abuse/Abuse001_x264/Abuse001_x264_7.mp4
8,Abuse,Abuse001_x264.mp4,Abuse/Abuse001_x264/Abuse001_x264_8.mp4


In [138]:
test_df = add_clip_paths(test_df)
val_df = add_clip_paths(val_df)

In [139]:
train_df['split'] = 'train'
test_df['split'] = 'test'
val_df['split'] = 'val'

all_annotations = pd.concat([train_df, test_df, val_df], ignore_index=True)

# Expringting all annotations to CSV
all_annotations.to_csv("../uca-dataset/uca_annotations.csv", index=False)

### Preparing UCA Dataset with Clipped Videos

In [28]:
def clip_videos_from_df(df, source_dir, output_dir):
    """
    Clips videos using MoviePy 2.x API based on 'start' and 'end' times in the DataFrame.
    Displays a progress bar and only logs failures or invalid clips.
    """
    os.makedirs(output_dir, exist_ok=True)
    total_videos = df['video'].nunique()

    print(f"🎬 Starting video clipping for {total_videos} unique videos ({len(df)} total clips)...\n")

    # Group videos first
    grouped_videos = list(df.groupby(["folder", "video"]))

    # tqdm progress bar
    for (folder, video_name), group in tqdm(grouped_videos, desc="Processing videos", unit="video"):
        src_path = os.path.join(source_dir, folder, video_name)
        base_name, ext = os.path.splitext(video_name)
        dest_subdir = os.path.join(output_dir, folder, base_name)
        os.makedirs(dest_subdir, exist_ok=True)

        if not os.path.exists(src_path):
            print(f"⚠️ Missing source video: {src_path}")
            continue

        try:
            video = VideoFileClip(src_path)
            for i, row in enumerate(group.itertuples(index=False)):
                start = float(row.start)
                end = min(float(row.end), video.duration)

                # Skip invalid segments
                if end <= start:
                    print(f"⏩ Skipping invalid segment ({start:.2f}-{end:.2f}) in {video_name}")
                    continue

                clip = video.subclipped(start, end)
                dest_path = os.path.join(dest_subdir, f"{base_name}_{i}.mp4")

                # Silent writing (no MoviePy output spam)
                clip.write_videofile(dest_path, audio=False, logger=None)

            video.close()

        except Exception as e:
            print(f"❌ Error processing {src_path}: {e}")

    print("\n✅ All videos processed successfully!")

In [89]:
source_dir = '../original-ucf/Videos'
destination_dir = '../uca-dataset'

clip_videos_from_df(
    df=train_df,
    source_dir=source_dir,         
    output_dir=destination_dir+"/train"
)

🎬 Starting video clipping for 1165 unique videos (15677 total clips)...



Processing videos:   0%|          | 0/1165 [00:00<?, ?video/s]

⚠️ Missing source video: ../Redo/Abuse/Abuse001_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse002_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse003_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse004_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse005_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse006_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse007_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse008_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse009_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse010_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse011_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse012_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse013_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse014_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse015_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse016_x264.mp4
⚠️ Missing source video: ../Redo/Abuse/Abuse017_x264.mp4
⚠️ Missing source video: ../Red

Processing videos:   4%|▍         | 48/1165 [00:18<07:19,  2.54video/s]

⚠️ Missing source video: ../Redo/Arrest/Arrest013_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest014_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest015_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest016_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest017_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest018_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest019_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest020_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest021_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest022_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest023_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest024_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest025_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest026_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest027_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest028_x264.mp4
⚠️ Missing source video: ../Redo/Arrest/Arrest029_x264.m

...

⚠️ Missing source video: ../Redo/Normal/Normal_Videos397_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos398_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos399_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos400_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos402_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos403_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos404_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos405_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos406_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos407_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos408_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos409_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos410_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos411_x264.mp4
⚠️ Missing source video: ../Redo/Normal/Normal_Videos412_x264.mp4
⚠️ Missing

Processing videos:  87%|████████▋ | 1016/1165 [09:04<00:56,  2.64video/s]

⚠️ Missing source video: ../Redo/Shooting/Shooting009_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting010_x264.mp4


Processing videos: 100%|██████████| 1165/1165 [09:06<00:00,  2.13video/s]

⚠️ Missing source video: ../Redo/Shooting/Shooting012_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting013_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting014_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting015_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting017_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting018_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting019_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting020_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting021_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting022_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting023_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting024_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting025_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting026_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting027_x264.mp4
⚠️ Missing source video: ../Redo/Shooting/Shooting028_x

In [31]:
source_dir = '../original-ucf/Videos'
destination_dir = '../uca-dataset'

clip_videos_from_df(
    df=val_df,
    source_dir=source_dir,         
    output_dir=destination_dir+"/val"
)

🎬 Starting video clipping for 379 unique videos (3534 total clips)...



Processing videos:   0%|          | 1/379 [00:02<14:50,  2.36s/video]/Users/samdenlepcha/miniconda3/envs/multimedia/lib/python3.10/site-packages/moviepy/video/io/ffmpeg_reader.py:190: UserWarning: In file ../original-ucf/Videos/Abuse/Abuse044_x264.mp4, 230400 bytes wanted but 0 bytes read at frame index 3940 (out of a total 3942 frames), at time 131.33/131.40 sec. Using the last valid frame instead.
  warnings.warn(
/Users/samdenlepcha/miniconda3/envs/multimedia/lib/python3.10/site-packages/moviepy/video/io/ffmpeg_reader.py:190: UserWarning: In file ../original-ucf/Videos/Abuse/Abuse044_x264.mp4, 230400 bytes wanted but 0 bytes read at frame index 3941 (out of a total 3942 frames), at time 131.37/131.40 sec. Using the last valid frame instead.
  warnings.warn(
Processing videos:   1%|▏         | 5/379 [00:07<06:52,  1.10s/video]/Users/samdenlepcha/miniconda3/envs/multimedia/lib/python3.10/site-packages/moviepy/video/io/ffmpeg_reader.py:190: UserWarning: In file ../original-ucf/Videos/A


✅ All videos processed successfully!


In [32]:
source_dir = '../original-ucf/Videos'
destination_dir = '../uca-dataset'

clip_videos_from_df(
    df=test_df,
    source_dir=source_dir,         
    output_dir=destination_dir+"/test"
)

🎬 Starting video clipping for 310 unique videos (4331 total clips)...



Processing videos:   3%|▎         | 8/310 [00:25<17:37,  3.50s/video]/Users/samdenlepcha/miniconda3/envs/multimedia/lib/python3.10/site-packages/moviepy/video/io/ffmpeg_reader.py:190: UserWarning: In file ../original-ucf/Videos/Arrest/Arrest046_x264.mp4, 230400 bytes wanted but 0 bytes read at frame index 7883 (out of a total 7884 frames), at time 262.77/262.81 sec. Using the last valid frame instead.
  warnings.warn(
Processing videos:  13%|█▎        | 40/310 [01:15<02:43,  1.65video/s]/Users/samdenlepcha/miniconda3/envs/multimedia/lib/python3.10/site-packages/moviepy/video/io/ffmpeg_reader.py:190: UserWarning: In file ../original-ucf/Videos/Assault/Assault035_x264.mp4, 230400 bytes wanted but 0 bytes read at frame index 897 (out of a total 900 frames), at time 29.90/30.00 sec. Using the last valid frame instead.
  warnings.warn(
/Users/samdenlepcha/miniconda3/envs/multimedia/lib/python3.10/site-packages/moviepy/video/io/ffmpeg_reader.py:190: UserWarning: In file ../original-ucf/Video


✅ All videos processed successfully!


#### Validating Data Prep

In [53]:
def check_for_duplicates(df, subset_cols=["video", "start", "end", "description"]):
    """
    Checks for duplicate video segments based on selected columns.
    Prints how many duplicates exist and which videos have them.
    """
    duplicated_rows = df[df.duplicated(subset=subset_cols, keep=False)]
    total_dupes = len(duplicated_rows)

    if total_dupes == 0:
        print("✅ No duplicates found.")
        return

    print(f"📊 Found {total_dupes} duplicate rows based on {subset_cols}")

    # Count duplicates per video
    dupe_counts = (
        duplicated_rows.groupby(["video", "start", "end"])
        .size()
        .sort_values(ascending=False)
        .reset_index(name="duplicate_count")
    )

    print("\n🎞️ Videos with duplicate timestamps:")
    display(dupe_counts.head(10))  # show top 10 by default

    return duplicated_rows

In [54]:
check_for_duplicates(test_df)

✅ No duplicates found.


In [55]:
check_for_duplicates(val_df)

✅ No duplicates found.


In [59]:
duplicated_rows = check_for_duplicates(train_df)
# These duplicates are present in th original transcripts as well. Leaving them for now

📊 Found 14 duplicate rows based on ['video', 'start', 'end', 'description']

🎞️ Videos with duplicate timestamps:


,video,start,end,duplicate_count
0,Arrest012_x264.mp4,58.5,64.3,2
1,Normal_Videos136_x264.mp4,550.2,558.5,2
2,Normal_Videos308_x264.mp4,31095.5,31150.2,2
3,Normal_Videos396_x264.mp4,2087.6,2095.5,2
4,Shooting008_x264.mp4,3.4,9.4,2
5,Shooting011_x264.mp4,101.6,111.6,2
6,Shooting011_x264.mp4,123.9,128.9,2


In [100]:
# This aligns perfectly with the UCA paper table 3
destination_dir = "../uca-dataset"
summarize_dataset_structure(base_dir=destination_dir)


📂 Stats for split: train
  🗂️ Abuse: 511 video clips
  🗂️ Arrest: 420 video clips
  🗂️ Arson: 123 video clips
  🗂️ Assault: 191 video clips
  🗂️ Burglary: 716 video clips
  🗂️ Explosion: 207 video clips
  🗂️ Fighting: 419 video clips
  🗂️ Normal: 10276 video clips
  🗂️ RoadAccidents: 441 video clips
  🗂️ Robbery: 722 video clips
  🗂️ Shooting: 378 video clips
  🗂️ Shoplifting: 470 video clips
  🗂️ Stealing: 562 video clips
  🗂️ Vandalism: 241 video clips
  ➤ Total video clips in 'train': 15677

📂 Stats for split: val
  🗂️ Abuse: 108 video clips
  🗂️ Arrest: 87 video clips
  🗂️ Arson: 53 video clips
  🗂️ Assault: 37 video clips
  🗂️ Burglary: 152 video clips
  🗂️ Explosion: 78 video clips
  🗂️ Fighting: 130 video clips
  🗂️ Normal: 2254 video clips
  🗂️ RoadAccidents: 44 video clips
  🗂️ Robbery: 199 video clips
  🗂️ Shooting: 29 video clips
  🗂️ Shoplifting: 126 video clips
  🗂️ Stealing: 164 video clips
  🗂️ Vandalism: 73 video clips
  ➤ Total video clips in 'val': 3534

📂 Stats for 

### YAML File Creation - Fine Tune SwinBERT

In [ ]:
def create_yaml_files(df, output_dir="../uca-dataset/"):
    """
    Converts UCA annotations CSV into SwinBERT-compatible YAML files for train/val/test.

    Args:
        csv_path (str): Path to CSV file with columns ['clip_path', 'description', 'split'].
        output_dir (str): Folder where YAMLs will be saved.
    """

    os.makedirs(output_dir, exist_ok=True)

    # Validate required columns
    required = {"clip_path", "description", "split"}
    if not required.issubset(df.columns):
        raise ValueError(f"CSV must contain columns: {required}")

    # Generate YAML for each split
    for split in df["split"].unique():
        split_df = df[df["split"] == split]
        split_yaml = {
            # This is based on file structure in Cloud GPU Instance
            f"{split}_videos": [os.path.join("../UCA-Dataset/w-captions", p) for p in split_df["clip_path"]],
            "captions": split_df["description"].tolist()
        }

        out_path = os.path.join(output_dir, f"{split}.yaml")
        with open(out_path, "w") as f:
            yaml.dump(split_yaml, f, default_flow_style=False, sort_keys=False, allow_unicode=True)

        print(f"✅ Saved: {out_path} ({len(split_df)} entries)")

In [ ]:
all_annotations = pd.read_csv("../uca-dataset/uca_annotations.csv")

In [146]:
all_annotations['description'] = all_annotations['description'].str.replace('\n', ' ').str.strip()

In [147]:
create_yaml_files(all_annotations)

✅ Saved: ../uca-dataset/train.yaml (15677 entries)
✅ Saved: ../uca-dataset/test.yaml (4331 entries)
✅ Saved: ../uca-dataset/val.yaml (3534 entries)
